Copyright (c) 2015, 2016 [Sebastian Raschka](sebastianraschka.com)

https://github.com/1iyiwei/pyml

[MIT License](https://github.com/1iyiwei/pyml/blob/master/LICENSE.txt)

# Python Machine Learning - Code Examples

# Chapter 8 - Applying Machine Learning To Sentiment Analysis

Let's apply what we have learned so far for a real case study.

Many people express opinions on the internet and social media sites.

Such opinions are a rich source of information for many applications:
* business
* politics
* science

Apply natural language processing (NLP), in particular sentiment analysis, over movie reviews

## Topics

Data-preprocessing
* cleaning and preparing text data from movie reviews
* building feature vectors from text documents

Training a machine learning model to classify positive and negative movie reviews

Working with large text datasets using out-of-core learning

<img src="./images/01_09.png" width=100%>

Note that the optional watermark extension is a small IPython notebook plugin that I developed to make the code reproducible. You can just skip the following line(s).

In [1]:
%load_ext watermark
%watermark -a '' -u -d -v -p numpy,pandas,matplotlib,sklearn,nltk

last updated: 2016-09-19 

CPython 3.5.2
IPython 4.2.0

numpy 1.11.1
pandas 0.18.1
matplotlib 1.5.1
sklearn 0.17.1
nltk 3.2.1


*The use of `watermark` is optional. You can install this IPython extension via "`pip install watermark`". For more information, please see: https://github.com/rasbt/watermark.*

### Overview

- [Obtaining the IMDb movie review dataset](#Obtaining-the-IMDb-movie-review-dataset)
- [Introducing the bag-of-words model](#Introducing-the-bag-of-words-model)
  - [Transforming words into feature vectors](#Transforming-words-into-feature-vectors)
  - [Assessing word relevancy via term frequency-inverse document frequency](#Assessing-word-relevancy-via-term-frequency-inverse-document-frequency)
  - [Cleaning text data](#Cleaning-text-data)
  - [Processing documents into tokens](#Processing-documents-into-tokens)
- [Training a logistic regression model for document classification](#Training-a-logistic-regression-model-for-document-classification)
- [Working with bigger data – online algorithms and out-of-core learning](#Working-with-bigger-data-–-online-algorithms-and-out-of-core-learning)
- [Summary](#Summary)

# Obtaining the IMDb movie review dataset

The [IMDB movie](http://www.imdb.com/) review set can be downloaded from [http://ai.stanford.edu/~amaas/data/sentiment/](http://ai.stanford.edu/~amaas/data/sentiment/).
* also available under [../datasets/movie/](../datasets/movie/) as part of the gibhub repo

50,000 movie reviews, manually labeled as being positive or negative for classification.

<hr>
### Note

If you have problems with creating the `movie_data.csv` file in the previous chapter, you can find a download a zip archive at 
https://github.com/1iyiwei/pyml/tree/master/code/datasets/movie
<hr>

In [2]:
import urllib.request
import os

# the file we eventually need to access
csv_filename = 'movie_data.csv'

# a global variable to select data source: local or remote
data_source = 'local'

if data_source == 'local':
    #basepath = '/Users/Sebastian/Desktop/aclImdb/'
    basepath = '../datasets/movie/'
    zip_filename = 'movie_data.csv.zip'
else: # remote
    url = 'http://ai.stanford.edu/~amaas/data/sentiment/'
    basepath = '.'
    zip_filename = 'aclImdb_v1.tar.gz'
    remote_file = os.path.join(url, zip_filename)
    local_file = os.path.join(basepath, zip_filename)
    csv_file = os.path.join(basepath, csv_filename)
    if not os.path.isfile(csv_file) and not os.path.isfile(local_file):
        urllib.request.urlretrieve(remote_file, local_file)

## Decompressing

After downloading the dataset, decompress the files.

A) If you are working with Linux or MacOS X, open a new terminal windowm `cd` into the download directory and execute 

`tar -zxf aclImdb_v1.tar.gz`

or
`tar -xvzf aclImdb_v1.tar.gz`
for the verbose mode

B) If you are working with Windows, download an archiver such as [7Zip](http://www.7-zip.org) to extract the files from the download archive.

C) The code below decompresses directly via Python.

In [3]:
# The code below decompresses directly via Python.
import os
import zipfile
import tarfile

# change the `basepath` to the directory of the
# unzipped movie dataset

csv_file = os.path.join(basepath, csv_filename)
zip_file = os.path.join(basepath, zip_filename)

if not os.path.isfile(csv_file):
    if tarfile.is_tarfile(zip_file):
        tartar = tarfile.open(zip_file, "r")
        #with tarfile.TarFile(zip_file, "r") as tartar:
        tartar.extractall(basepath)
        tartar.close()
    else:
        with zipfile.ZipFile(zip_file, "r") as zipper:
            zipper.extractall(basepath)
            zipper.close()

## Reading the dataset

The decompressed file is in csv format, we can read it via panda as usual.

PyPrind (Python Progress Indicator)
* useful for visualizing progress for processing large datasets
* pip install pyprind


### Compatibility Note:

I received an email from a reader who was having troubles with reading the movie review texts due to encoding issues. Typically, Python's default encoding is set to `'utf-8'`, which shouldn't cause troubles when running this IPython notebook. You can simply check the encoding on your machine by firing up a new Python interpreter from the command line terminal and execute

    >>> import sys
    >>> sys.getdefaultencoding()
    
If the returned result is **not** `'utf-8'`, you probably need to change your Python's encoding to `'utf-8'`, for example by typing `export PYTHONIOENCODING=utf8` in your terminal shell prior to running this IPython notebook. (Note that this is a temporary change, and it needs to be executed in the same shell that you'll use to launch `ipython notebook`.

Alternatively, you can replace the lines 

    with open(os.path.join(path, file), 'r') as infile:
    ...
    pd.read_csv('./movie_data.csv')
    ...
    df.to_csv('./movie_data.csv', index=False)

by 

    with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
    ...
    pd.read_csv('./movie_data.csv', encoding='utf-8')
    ...
    df.to_csv('./movie_data.csv', index=False, encoding='utf-8')
    
in the following cells to achieve the desired effect.

In [4]:
import pyprind
import pandas as pd
import os

db_path = 'aclImdb';

if not os.path.isfile(csv_file):
    labels = {'pos': 1, 'neg': 0}
    pbar = pyprind.ProgBar(50000)
    df = pd.DataFrame()
    for s in ('test', 'train'):
        for l in ('pos', 'neg'):
            path = os.path.join(db_path, s, l)
            for file in os.listdir(path):
                with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                    txt = infile.read()
                df = df.append([[txt, labels[l]]], ignore_index=True)
                pbar.update()
    df.columns = ['review', 'sentiment']

Optional: Saving the assembled data as CSV file:

In [5]:
if not os.path.isfile(csv_file):
    df.to_csv(os.path.join(basepath, csv_filename), index=False, encoding='utf-8')

Read back the data-frame from file, local or remote.

In [6]:
import pandas as pd

df = pd.read_csv(os.path.join(basepath, csv_filename), encoding='utf-8')

Shuffling the DataFrame:

In [7]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [8]:
df.head(3)

,review,sentiment
11841,"Election is a Chinese mob movie, or triads in ...",1
19602,I was just watching a Forensic Files marathon ...,0
45519,Police Story is a stunning series of set piece...,1


# Introducing the bag-of-words model

We need to convert the dataset into numerical form
* e.g. categorical variables (nominal or ordinal) into numerical variables

Bag-of-words: represent text as numerical feature vectors
* create a vocabulary of unique tokens, e.g. words
* compute a histogram counting the number of occurances of each word

The feature vector would be sparse since most of the entries are $0$

## Transforming documents into feature vectors

By calling the fit_transform method on CountVectorizer, we just constructed the vocabulary of the bag-of-words model and transformed the following three sentences into sparse feature vectors:
1. The sun is shining
2. The weather is sweet
3. The sun is shining, the weather is sweet, and one and one is two


In [9]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

Print the contents of the vocabulary to get a better understanding of the underlying concepts:

In [10]:
print(count.vocabulary_)

{'and': 0, 'shining': 3, 'sweet': 5, 'sun': 4, 'is': 1, 'two': 7, 'weather': 8, 'one': 2, 'the': 6}


The vocabulary is stored in a Python dictionary
* key: words
* value: integer indices

In [11]:
# convert from sparse dictionary to dense array
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


Each index position in the feature vectors shown here corresponds to the integer values that are stored as dictionary items in the CountVectorizer vocabulary. 

For example, the 1st feature at index position 0 resembles the count of the word and, which only occurs in the last document, and the word is at index position 1 (the 2nd feature in the document vectors) occurs in all three sentences. 

Those values in the feature vectors are also called the raw term frequencies: *tf (t,d)*—the number of times a term t occurs in a document *d*.

# N-gram

N contiguous sequence of items

1-gram: individual words
* e.g. the, sun, is, shining

2-gram: pairs of adjacent words
* e.g. the sun, sun is, is shining

CountVectorizer can work with n-gram via the ngram_range parameter.

## Assessing word relevancy via term frequency-inverse document frequency

Term-frequency (tf) alone is not enough.
* common words typically don't contain useful or discriminatory information.
* e.g., the, is, and ...

Also consider inverse document frequency (idf)
* downweight those frequently occurring words in the feature vectors. 

The tf-idf can be defined as the product of the term frequency and the inverse document frequency:
$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

Here the tf(t, d) is the term frequency introduced above,
and the inverse document frequency $idf(t, d)$ can be calculated as:
$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)}$$
* $n_d$ is the total number of documents
* $df(d, t)$ is the number of documents $d$ that contain the term $t$. 

$idf$ gives higher weights to rarer words.

Note
* adding the constant 1 to the denominator to avoid division-by-zero.
* the log is used to ensure that low document frequencies are not given too much weight.

Scikit-learn implements yet another transformer, the `TfidfTransformer`, that takes the raw term frequencies from `CountVectorizer` as input and transforms them into tf-idfs:

In [12]:
np.set_printoptions(precision=2)

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.    0.56  0.56  0.    0.43  0.    0.  ]
 [ 0.    0.43  0.    0.    0.    0.56  0.43  0.    0.56]
 [ 0.5   0.45  0.5   0.19  0.19  0.19  0.3   0.25  0.19]]


As we saw in the previous subsection, the word 'is' had the largest term frequency in the 3rd document, being the most frequently occurring word. 

However, after transforming the same feature vector into tf-idfs, we see that the word is is
now associated with a relatively small tf-idf (0.45) in document 3 since it is
also contained in documents 1 and 2 and thus is unlikely to contain any useful, discriminatory information.


# Scikit-learn tf-idf

However, if we'd manually calculated the tf-idfs of the individual terms in our feature vectors, we'd have noticed that the `TfidfTransformer` calculates the tf-idfs slightly differently compared to the standard textbook equations that we defined earlier.

The equations for the idf and tf-idf that were implemented in scikit-learn are:

$$\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$$

The tf-idf equation that was implemented in scikit-learn is as follows:

$$\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)$$

While it is also more typical to normalize the raw term frequencies before calculating the tf-idfs, the `TfidfTransformer` normalizes the tf-idfs directly.

By default (`norm='l2'`), scikit-learn's TfidfTransformer applies the L2-normalization, which
returns a vector of length 1 by dividing an un-normalized feature vector *v* by its L2-norm:

$$v_{\text{norm}} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}} = \frac{v}{\big (\sum_{i=1}^{n} v_{i}^{2}\big)^\frac{1}{2}}$$

## Example

To make sure that we understand how TfidfTransformer works, let us walk
through an example and calculate the tf-idf of the word is in the 3rd document.

The word `is` has a term frequency of 3 (tf = 3) in document 3, and the document frequency of this term is 3 since the term is occurs in all three documents (df = 3). Thus, we can calculate the idf as follows:

$$\text{idf}("is", d3) = log \frac{1+3}{1+3} = 0$$

Now in order to calculate the tf-idf, we simply need to add 1 to the inverse document frequency and multiply it by the term frequency:

$$\text{tf-idf}("is",d3)= 3 \times (0+1) = 3$$

In [14]:
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf of term "is" = %.2f' % tfidf_is)

tf-idf of term "is" = 3.00


If we repeated these calculations for all terms in the 3rd document, we'd obtain the following tf-idf vectors: [3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]. 

However, we notice that the values in this feature vector are different from the values that we obtained from the TfidfTransformer that we used previously.

The final step that we are missing in this tf-idf calculation is the L2-normalization, which can be applied as follows:

$$\text{tfi-df}_{norm} = \frac{[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]}{\sqrt{[3.39^2, 3.0^2, 3.39^2, 1.29^2, 1.29^2, 1.29^2, 2.0^2 , 1.69^2, 1.29^2]}}$$

$$=[0.5, 0.45, 0.5, 0.19, 0.19, 0.19, 0.3, 0.25, 0.19]$$

$$\Rightarrow \text{tfi-df}_{norm}("is", d3) = 0.45$$

As we can see, the results match the results returned by scikit-learn's `TfidfTransformer` (below).

In [15]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True) # notice norm is None not l2
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1] # for the last document
raw_tfidf 

array([ 3.39,  3.  ,  3.39,  1.29,  1.29,  1.29,  2.  ,  1.69,  1.29])

In [16]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([ 0.5 ,  0.45,  0.5 ,  0.19,  0.19,  0.19,  0.3 ,  0.25,  0.19])

## Cleaning text data

The text may contain stuff irrelevant for sentiment analysis.
* html tags, punctuation, non-letter characters, etc.

In [17]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

### Use regular expression for cleaning text data

References:
* https://developers.google.com/edu/python/regular-expressions
* https://docs.python.org/3.4/library/re.html

Remove all punctuations except for emoticons which convey sentiments.

In [18]:
import re
def preprocessor(text):
    
    # [] for set of characters, ^ inside [] means invert, i.e. not > below
    # * means 0 or more occurances of the pattern
    text = re.sub('<[^>]*>', '', text) # remove html tags between pairs of < and >
    
    # () for group, subpart of the whole pattern we look for
    # findall will return tuples each containing groups
    # (?:) means not returing the group result for findall
    # | means or, \ for escape sequence
    # first group eye : or ; or =
    # second group nose - 0 or 1 time via ?
    # third group mouth ) or ( or D or P 
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # matching examples:
    # :-)
    # =D
    
    # convert to lower case as upper/lower case doesn't matter for sentiment
    # replace all non-word characters by space
    # \w: letters, digits, _
    # \W: the complement set
    text = re.sub('[\W]+', ' ', text.lower())
    
    # add back emoticons, though in different orders 
    # and without nose "-", e.g. :) and :-) are considered the same
    text = text + ' '.join(emoticons).replace('-', '')
        
    return text

### Example results

In [19]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [20]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

Emotions are moved to the end; ordering doesn't matter for 1-gram analysis.

### Cleanup the data

In [21]:
df['review'] = df['review'].apply(preprocessor)

## Processing documents into tokens

Split an entity into constituting components, e.g. words for documents.

Stemming: transform a word into root form.
* e.g. running $\rightarrow$ run
* see http://www.nltk.org/book/ for more details and options for stemming.

In [22]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    # split along white spaces
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [23]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [24]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

## Remove stop-words

Stop-words are extremely common in all texts
* e.g. is, and, has, etc.

Remove them completely can help document analysis.

In [25]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\liyiwei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# Training a logistic regression model for document classification

Let's try to apply logistic regression to classify the movie reviews.

Use cleaned-up documents (no html tags or punctuations except for emoticons), but leave tokenization as hyper-parameters.


## Split into training and test datasets

In [27]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [28]:
# Use a smaller subset if it took too long to run the full datasets above
train_subset_size = 2500
test_subset_size = 2500

#print(X_train.shape)

if train_subset_size > 0:
    X_train = X_train[:train_subset_size]
    y_train = y_train[:train_subset_size]
    
if test_subset_size > 0:
    X_test = X_test[:test_subset_size]
    y_test = y_test[:test_subset_size]
    
#print(X_train.shape)    

## Grid-search hyper-parameters

Two grid sets: with and without idf

Different regularization strengths via $C$.

Use pipeline as before.

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=1)

In [ ]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:  3.5min
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed: 16.0min
[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed: 19.2min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'clf__penalty': ['l1', 'l2'], 'vect__stop_words': [['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 't...': [<function tokenizer at 0x00000259933FB0D0>, <function tokenizer_porter at 0x00000259933FB8C8>]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=1)

In [ ]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__penalty': 'l2', 'vect__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too'

The CV accuracy and test accuracy would be a bit lower if we use a subset of all data, but are still reasonable.

In [ ]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.843


####  Start comment:
    
Please note that `gs_lr_tfidf.best_score_` is the average k-fold cross-validation score. I.e., if we have a `GridSearchCV` object with 5-fold cross-validation (like the one above), the `best_score_` attribute returns the average score over the 5-folds of the best model. To illustrate this with an example:

In [ ]:
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np

np.random.seed(0)
np.set_printoptions(precision=6)
y = [np.random.randint(3) for i in range(25)]
X = (y + np.random.randn(25)).reshape(-1, 1)

cv5_idx = list(StratifiedKFold(y, n_folds=5, shuffle=False, random_state=0))
cross_val_score(LogisticRegression(random_state=123), X, y, cv=cv5_idx)

array([ 0.6,  0.4,  0.6,  0.2,  0.6])

By executing the code above, we created a simple data set of random integers that shall represent our class labels. Next, we fed the indices of 5 cross-validation folds (`cv3_idx`) to the `cross_val_score` scorer, which returned 5 accuracy scores -- these are the 5 accuracy values for the 5 test folds.  

Next, let us use the `GridSearchCV` object and feed it the same 5 cross-validation sets (via the pre-generated `cv3_idx` indices):

In [ ]:
from sklearn.grid_search import GridSearchCV
gs = GridSearchCV(LogisticRegression(), {}, cv=cv5_idx, verbose=3).fit(X, y) 

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV] ....................................... , score=0.600000 -   0.0s
[CV]  ................................................................
[CV] ....................................... , score=0.400000 -   0.0s
[CV]  ................................................................
[CV] ....................................... , score=0.600000 -   0.0s
[CV]  ................................................................
[CV] ....................................... , score=0.200000 -   0.0s
[CV]  ................................................................
[CV] ....................................... , score=0.600000 -   0.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


As we can see, the scores for the 5 folds are exactly the same as the ones from `cross_val_score` earlier.

Now, the best_score_ attribute of the `GridSearchCV` object, which becomes available after `fit`ting, returns the average accuracy score of the best model:

In [ ]:
gs.best_score_

0.47999999999999998

As we can see, the result above is consistent with the average score computed the `cross_val_score`.

In [ ]:
cross_val_score(LogisticRegression(), X, y, cv=cv5_idx).mean()

0.47999999999999998

#### End comment.

<hr>
<hr>

# Naive Bayes

Popular for text classification, e.g. spam filtering.
* easy to implement
* fast to compute
* good performance with small datasets

See http://sebastianraschka.com/Articles/2014_naive_bayes_1.html for more details.

# Working with bigger data - online algorithms and out-of-core learning

The grid-search in the previous section is quite computationally expensive.

But real world datasets can be much larger!

Out-of-core learning can help us deal with large datasets without super-computers.

SGDClassifier: stochastic gradient descent classifier

In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized


def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [ ]:
next(stream_docs(path=csv_file))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

### Python generators

http://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do-in-python


In [ ]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

## Out-of-core Vectorizer

CountVectorizer holds complete vocabulary in memory

TfidfVectorizer keeps all training data in memory

[HashVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) comes for rescue
* hash words into histogram bins
* can have collision, but with low probability
* collision reduces histogram resolution, but still suffices for classification and can reduce number of features and thus over-fitting

## Hash

A function that maps items into cells in a hash table.
* easy/fast to compute
* can have collision, i.e. different items map into the same hash entry
* try to minimize and/or handle collision

<a href="https://en.wikipedia.org/wiki/Hash_function">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/58/Hash_table_4_1_1_0_0_1_0_LL.svg/300px-Hash_table_4_1_1_0_0_1_0_LL.svg.png">
</a>


In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21, # large enough to minimize has collision
                         preprocessor=None, 
                         tokenizer=tokenizer)

# logistic regression for loss
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path=csv_file)

## Start out-of-core learning

Training: 45,000 samples

Test: 5,000 samples

In [ ]:
# full size
num_batches = 45
batch_size = 1000
test_size = 5000

In [ ]:
# subset if the fullset took too long to run
batch_size = 100
test_size = 500

In [ ]:
import pyprind

pbar = pyprind.ProgBar(num_batches)

classes = np.array([0, 1])
for _ in range(num_batches):
    X_train, y_train = get_minibatch(doc_stream, size=batch_size)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:06


In [ ]:
X_test, y_test = get_minibatch(doc_stream, size=test_size)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.842


In [ ]:
clf = clf.partial_fit(X_test, y_test)

# Summary

A case study of sentiment analysis of movie reviews

Data preprocessing is very important!
* representation - documents into bag of words via tf-idf
* cleaning, stemming, tokenization, etc.

Deal with large datasets
* streaming 

# Reading

PML Chapter 8

[word2vec](https://code.google.com/archive/p/word2vec/)